In [32]:
import json
import pandas as pd

file_path = '/home/ahmed/Desktop/AGH_Data_Agency_Holding_SA/Projet_LLM_Server/data_handling/quiz_complet.json'

try:
    # Utilisation de 'with open' pour ouvrir et lire le fichier de manière sécurisée
    with open(file_path, 'r', encoding='utf-8') as file:
        # La fonction json.load() lit le contenu du fichier et le convertit
        # en une structure de données Python (liste ou dictionnaire).
        data = json.load(file)
    
    print("✅ Fichier JSON chargé avec succès.")
    print("--- Type de données en Python ---")
    print(type(data))
    print("--- Aperçu des données ---")
except FileNotFoundError:
    print(f"❌ Erreur : Le fichier '{file_path}' n'a pas été trouvé.")
except json.JSONDecodeError:
    print(f"❌ Erreur : Le fichier '{file_path}' contient un JSON invalide.")
except Exception as e:
    print(f"❌ Une erreur inattendue s'est produite : {e}")
# 1. Charger les données JSON


# Liste pour stocker les lignes aplaties
flattened_data = []

# 2. Parcourir les données
for surah_item in data:
    surah_title_en = surah_item["title"]["en"]
    surah_level = surah_item["level"]
    
    # Parcourir chaque question dans la liste 'questions'
    for question_item in surah_item["questions"]:
        
        # Initialiser les valeurs par défaut pour les champs optionnels (comme options.en)
        question_type = question_item.get("type")
        question_en = question_item["question"].get("en")
        
        # 'options' et 'correct_option_index' sont spécifiques aux questions 'multiple_choice'
        options_en = question_item.get("options", {}).get("en")
        correct_option_index = question_item.get("correct_option_index")
        
        # Si c'est une question 'true_false', l'index correct n'existe pas, 
        # mais la réponse est stockée dans 'answer'. On peut la convertir.
        if question_type == "true_false":
            # Pour l'affichage, on utilise une représentation simple
            correct_index_display = f"Answer: {question_item.get('answer')}"
            options_display = ["True", "False"] 
            # Note: Vous pourriez vouloir stocker "True" ou "False" comme l'option correcte
            # si vous voulez un seul champ 'options' et 'correct_option_index' pour tout.
        else:
            correct_index_display = correct_option_index
            options_display = options_en

        # Créer l'objet de ligne aplatie
        flattened_row = {
            "title.en": surah_title_en,
            "level": surah_level,
            "questions.type": question_type,
            "question.en": question_en,
            "options.en": options_display,
            "correct_option_index": correct_index_display
        }
        
        # Ajouter la ligne à la liste
        flattened_data.append(flattened_row)

# 3. Afficher les résultats (en utilisant pandas pour une belle sortie)
df = pd.DataFrame(flattened_data)

print("--- Résultat sous forme de DataFrame (sortie tabulaire) ---")
print(df)

# Optionnel: Exporter au format CSV
# df.to_csv("surah_questions.csv", index=False)

✅ Fichier JSON chargé avec succès.
--- Type de données en Python ---
<class 'list'>
--- Aperçu des données ---
--- Résultat sous forme de DataFrame (sortie tabulaire) ---
          title.en     level   questions.type  \
0    Surah Al-Isra  advanced  multiple_choice   
1    Surah Al-Isra  advanced       true_false   
2    Surah Al-Isra  advanced  multiple_choice   
3    Surah Al-Isra  advanced       true_false   
4    Surah Al-Isra  advanced  multiple_choice   
..             ...       ...              ...   
630   Surah Maryam  advanced  multiple_choice   
631   Surah Maryam  advanced       true_false   
632   Surah Maryam  advanced  multiple_choice   
633   Surah Maryam  advanced       true_false   
634   Surah Maryam  advanced  multiple_choice   

                                           question.en  \
0                 What is the main theme of the surah?   
1                       The surah is Meccan in nature.   
2                    How many verses are in the surah?   
3    The

In [34]:
df.sample(5)

,title.en,level,questions.type,question.en,options.en,correct_option_index
341,Surah Nuh,intermediate,true_false,The surah focuses on Nuh’s call to his people ...,"[True, False]",Answer: True
598,Surah Quraysh,advanced,true_false,The surah encourages gratitude to Allah for Hi...,"[True, False]",Answer: True
604,Surah Al-Infitar,advanced,multiple_choice,What is the purpose of describing cosmic event...,[Remind people of Allah’s greatness and accoun...,0
232,Surah At-Tariq,advanced,multiple_choice,How many verses does Surah At-Tariq have?,"[17, 19, 15, 18]",0
558,Surah Al-Qamar,advanced,true_false,The surah contains the story of Noah and his p...,"[True, False]",Answer: True


In [35]:
df = df.rename(columns={
    'title.en': 'surah_name',
    'level': 'level',
    'questions.type': 'questions_type',
    'question.en': 'question_text',
    'options.en': 'options',
    'correct_option_index': 'correct_option'
})

In [36]:
df.sample(2)

,surah_name,level,questions_type,question_text,options,correct_option
302,Surah An-Nisa – Part 3,advanced,multiple_choice,What is the main theme of Part 3?,"[Jihad rulings, Marriage rulings only, Zakat r...",0
17,Surah Al-Waqia,advanced,multiple_choice,Which precise description of punishment is men...,"[Blazing fire, burning food, Whipping, Isolati...",0


In [ ]:
df.to_json(
    'quizzes_handling.json',  # Nommez votre fichier de sortie
    orient='records',              # Indique que chaque objet du JSON est une ligne du DataFrame
    indent=4,                      # Pour rendre le fichier lisible avec une belle indentation
    force_ascii=False              # Important pour conserver les caractères arabes ou spéciaux si votre DataFrame en contient
)